In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!unzip /content/gdrive/MyDrive/dataset_3.zip

In [3]:
# Каталог с данными для обучения
train_dir = '/content/dataset_3/train'
# Каталог с данными для проверки
val_dir = '/content/dataset_3/validation'
# Каталог с данными для тестирования
test_dir = '/content/dataset_3/test'
# Размеры изображения
img_width, img_height = 50, 50
# Размерность тензора на основе изображения для входных данных в нейронную сеть
input_shape = (img_width, img_height, 3)
# Количество эпох
epochs = 80
# Размер мини-выборки
batch_size = 32
#Количество изображений для обучения
nb_train_samples = 4761
# Количество изображений для проверки
nb_validation_samples = 345
# Количество изображений для тестирования
nb_test_samples = 345

In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.models import model_from_json
from keras.models import load_model
import matplotlib.pyplot as plt
from keras.optimizers import Nadam
import numpy as np


from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
# Загружаем свёрточные слои VGG16 без возможности изменения весов

conv = VGG16(include_top=False, input_shape=input_shape)
for layer in conv.layers:
	layer.trainable = False
#Добавляем полносвязные слои
model = Sequential()

model.add(conv)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

#Компилируем модель
model.compile(loss="categorical_crossentropy",
				optimizer=Nadam(lr=1e-4),
				metrics=["accuracy"])



In [ ]:
# Инициализируем ImageDataGenerator для аугментации изображений 
from keras. preprocessing. image import ImageDataGenerator
datagen = ImageDataGenerator( featurewise_center=False,samplewise_center=False,featurewise_std_normalization=False,
                              samplewise_std_normalization = False,zca_whitening=False,rotation_range=0.5,
                              zoom_range=0.5,width_shift_range=0.5,height_shift_range=0.5,horizontal_flip=True,rescale=1. / 255, vertical_flip=True)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size =batch_size,
    class_mode='categorical')

Found 1035 images belonging to 3 classes.


In [ ]:


train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


# Запускаем тренировочный процесс
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps= (nb_validation_samples // batch_size))

# Сохраняем структуру сети и веса нейронов
#model_json = model.to_json()
with open("VGG_GarbageClassifier_RGB.json", "w") as json_file :
	json_file.write(model_json)

model.save_weights("VGG16_GarbageClassifier_RGB5.h5")
print("Saved model to disk")

model.save('VGG_GarbageClassifier_RGB.model')

#Рисуем диаграмму точности по мере обучения
print(history.history.keys())
plt.figure(1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

In [ ]:
# Оцениваем качество обучения модели на тестовой выборке 
model.load_weights('VGG16_GarbageClassifier_RGB5.h5')
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print(scores[1]*100)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


79.6875


In [ ]:
#Проверим работу сети на изображении металической консервной банки, взятой из интеренета    
import cv2
CATEGORIES = ["glass","metal","plastic"]
#Функция, подготавливающая изображение к подаче в нейросеть
def prepare(file):
    IMG_SIZE = 50
    img_array = cv2.imread(file)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
model = tf.keras.models.load_model("VGG_GarbageClassifier_RGB.model")
image = "/content/gdrive/MyDrive/matal1.jpg" 
image = prepare(image)
prediction = model.predict([image])
prediction = list(prediction[0])
for i in range(len(CATEGORIES)):
  print(CATEGORIES[i],prediction[i]*100)


glass 0.05012294859625399
metal 79.97218370437622
plastic 19.977685809135437
